In [13]:
import cv2
import numpy as np
import os
from IPython.display import Image
from IPython.display import display
from matplotlib import pyplot as plt

#Video reading my cv2 stopped working since ffmpeg has some issue on my system. So using skvideo to read.
from skvideo.io import VideoCapture

'''Folder with input videos in it'''
folder = "/Users/homw/Documents/MSDS16/IndStudy/handwash/"

#Extract all the foreground pixels assuming only handwash activity in the video
def get_fg_from_video(file_path):
    vid = VideoCapture(file_path)
    ''' Get frames per second (fps) to calculate duration'''
    #fps    = vid.get(cv2.cv.CV_CAP_PROP_FPS)
    fps = 10
    ''' Initializing parameters'''
    #fcount = 1
    fgscore = []
    '''Read the video file and prepare a ref frame'''
    success, frame0 = vid.read()
    frame0 = cv2.cvtColor(frame0, cv2.COLOR_RGB2GRAY)
    '''Required for erode operation to cleanup the noise'''
    kernel = np.ones((5,5), np.uint8)
    while success:
        success, frame1 = vid.read()

        if success:

            frame1 = cv2.cvtColor(frame1, cv2.COLOR_RGB2GRAY)
            #Frame differencing and threshold to capture foreground
            diff = cv2.absdiff(frame1, frame0)
            _,diff = cv2.threshold(diff,80,255,cv2.THRESH_BINARY)

            #Erode the image to avoid false positives with 5X5 kernel
            diff = cv2.erode(diff, kernel, iterations=1)
            fgscore.append(np.count_nonzero(diff))
            #fcount += 1
            if cv2.waitKey(10) == 27:
                break
    return fgscore, fps


#Calculates and prints the handwash details from the video
def print_handwash_details(fgscore, fps, file_name):
    #handwash_totframes = len([f for f in fgscore if f > 1000])
    handwash_startframe = next((index for index, f in enumerate(fgscore) if f > 1000), -1)
    handwash_endframe = next((index for index, f in enumerate(fgscore[::-1]) if f > 1000), -1)
    handwash_endframe = len(fgscore) - handwash_endframe
    handwash_totframes = handwash_endframe-handwash_startframe
    
    handwash_len_seconds = handwash_totframes/fps
    print("handwash start frame is %d and end frame is %d" %(handwash_startframe, handwash_endframe))
    print("Total length of handwash is %01f seconds for %s" %(handwash_len_seconds, file_name))
    

#Main code
#files = next(os.walk(folder))[2]
files = os.listdir(folder)[1:]
for f in files:
    file_path = os.path.join(folder, f)
    foreground_score, fps = get_fg_from_video(file_path)
    print_handwash_details(foreground_score, fps, f)

handwash start frame is 76 and end frame is 572
Total length of handwash is 49.000000 seconds for 3132 - Fri Jan 15 15-47-38 2016.mp4
handwash start frame is 29 and end frame is 212
Total length of handwash is 18.000000 seconds for 3135 - Fri Jan 15 11-23-35 2016.mp4
handwash start frame is 70 and end frame is 250
Total length of handwash is 18.000000 seconds for 3135 - Fri Jan 15 18-43-35 2016.mp4
handwash start frame is 109 and end frame is 335
Total length of handwash is 22.000000 seconds for 3135 - Sat Jan 16 09-41-55 2016.mp4


In [12]:
file_path

'/Users/homw/Documents/MSDS16/IndStudy/handwash/3132 - Fri Jan 15 15-47-38 2016.mp4'